<a href="https://colab.research.google.com/github/zcongfly/huggingface-nlp-learning-note/blob/main/08_%E4%BD%BF%E7%94%A8_Trainer_API_%E6%88%96%E8%80%85_Keras_%E5%BE%AE%E8%B0%83%E4%B8%80%E4%B8%AA%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用 Trainer API 或者 Keras 微调一个模型

In [ ]:
# Install the Transformers, Datasets, and Evaluate libraries to run this notebook.
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

Transformers提供了一个 Trainer 类来帮助您在自己的数据集上微调任何预训练模型。完成上一节中的所有数据预处理工作后，您只需要执行几个步骤来创建 Trainer .最难的部分可能是为 Trainer.train()配置运行环境，因为它在 CPU 上运行速度会非常慢。如果您没有设置 GPU，您可以访问免费的 GPU 或 TPU[Google Colab](https://colab.research.google.com/).

下面的示例假设您已经执行了上一节中的示例。下面这段代码，概括了您需要提前运行的代码：

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

## Training

在我们定义我们的 Trainer 之前首先要定义一个 TrainingArguments 类，它将包含 Trainer用于训练和评估的所有超参数。您唯一必须提供的参数是保存训练模型的目录，以及训练过程中的检查点。对于其余的参数，您可以保留默认值，这对于基本微调应该非常有效。

如果您想在训练期间自动将模型上传到 Hub，请将push_to_hub=True添加到TrainingArguments之中. 我们将在第四章中详细介绍这部分。

In [2]:
import accelerate
print(accelerate.__version__)

0.20.3


下面这段代码需要开GPU才能成功运行，否则会一直报错。

我理解是TrainingArguments需要用到accelerate这个库，然而这个库依赖于GPU来加速。所以运行这章代码记得把colab的GPU加速打开。

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

第二步是定义我们的模型。正如在之前的章节一样，我们将使用 AutoModelForSequenceClassification 类，它有两个参数：

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

你会注意到，和第二章不一样的是，在实例化此预训练模型后会收到警告。这是因为 BERT 没有在句子对分类方面进行过预训练，所以预训练模型的头部已经被丢弃，而是添加了一个适合句子序列分类的新头部。警告表明一些权重没有使用（对应于丢弃的预训练头的那些），而其他一些权重被随机初始化（新头的那些）。最后鼓励您训练模型，这正是我们现在要做的。

一旦我们有了我们的模型，我们就可以定义一个 Trainer 通过将之前构造的所有对象传递给它——我们的model 、training_args ，训练和验证数据集，data_collator ，和 tokenizer ：

In [8]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

请注意，当您在这里完成tokenizer后，默认 Trainer使用 的data_collator会使用之前预定义的 DataCollatorWithPadding ，因此您可以在这个例子中跳过 data_collator=data_collator。在第 2 节中向您展示这部分处理仍然很重要！

为了让预训练模型在在我们的数据集上微调，我们只需要调用Trainer的train() 方法 ：

In [9]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.504600
1000,0.295400


TrainOutput(global_step=1377, training_loss=0.3267569254512967, metrics={'train_runtime': 195.6565, 'train_samples_per_second': 56.241, 'train_steps_per_second': 7.038, 'total_flos': 406183858377360.0, 'train_loss': 0.3267569254512967, 'epoch': 3.0})

这将开始微调（在GPU上应该需要几分钟），并每500步报告一次训练损失。但是，它不会告诉您模型的性能如何（或质量如何）。这是因为:

1. 我们没有通过将evaluation_strategy设置为“steps”(在每次更新参数的时候评估)或“epoch”(在每个epoch结束时评估)来告诉Trainer在训练期间进行评估。
2. 我们没有为Trainer提供一个compute_metrics()函数来直接计算模型的好坏(否则评估将只输出loss，这不是一个非常直观的数字)。

## 评估

让我们看看如何构建一个有用的 compute_metrics() 函数并在我们下次训练时使用它。该函数必须采用 EvalPrediction 对象（带有 predictions 和 label_ids 字段的参数元组）并将返回一个字符串到浮点数的字典（字符串是返回的指标的名称，而浮点数是它们的值）。我们可以使用 Trainer.predict() 命令来使用我们的模型进行预测：

In [10]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


predict() 方法是具有三个字段的命名元组： predictions , label_ids ， 和 metrics .这 metrics 字段将只包含传递的数据集的loss，以及一些运行时间（预测所需的总时间和平均时间）。如果我们定义了自己的 compute_metrics() 函数并将其传递给 Trainer ，该字段还将包含compute_metrics() 的结果。如你看到的， predictions 是一个形状为 408 x 2 的二维数组（408 是我们使用的数据集中元素的数量）。这些是我们传递给predict()的数据集的每个元素的结果(logits)（正如你在之前的章节看到的情况）。要将我们的预测的可以与真正的标签进行比较，我们需要在第二个轴上取最大值的索引：

In [11]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

现在建立我们的 compute_metric() 函数来较为直观地评估模型的好坏，我们将使用Evaluate 库中的指标。我们可以像加载数据集一样轻松加载与 MRPC 数据集关联的指标，这次使用 evaluate.load() 函数。返回的对象有一个 compute()方法我们可以用来进行度量计算的方法：

In [12]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8578431372549019, 'f1': 0.9006849315068494}

您获得的确切结果可能会有所不同，因为模型头的随机初始化可能会影响最终建立的模型。在这里，我们可以看到我们的模型在验证集上的准确率为 85.78%，F1 分数为 89.97。这是用于评估 GLUE 基准的 MRPC 数据集结果的两个指标。而在BERT 论文中展示的基础模型的 F1 分数为 88.9。那是 uncased 模型，而我们目前正在使用 cased 模型，通过改进得到了更好的结果。

最后将所有东西打包在一起，我们得到了我们的 compute_metrics() 函数：

In [13]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

为了查看模型在每个训练周期结束的好坏，下面是我们如何使用compute_metrics()函数定义一个新的 Trainer ：

In [14]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

请注意，我们设置了了一个新的 TrainingArguments 它的evaluation_strategy 设置为 epoch 并创建了一个新模型。如果不创建新的模型就直接训练，就只会继续训练之前我们已经训练过的模型。要启动新的训练运行，我们执行：

In [15]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.395915,0.823529,0.871886
2,0.515000,0.489869,0.852941,0.898990
3,0.315700,0.634946,0.862745,0.903114


TrainOutput(global_step=1377, training_loss=0.34340206747394414, metrics={'train_runtime': 193.1021, 'train_samples_per_second': 56.985, 'train_steps_per_second': 7.131, 'total_flos': 406183858377360.0, 'train_loss': 0.34340206747394414, 'epoch': 3.0})

这一次，它将在训练loss之外，还会输出每个 epoch 结束时的验证loss和指标。同样，由于模型的随机头部初始化，您达到的准确率/F1 分数可能与我们发现的略有不同，但它应该在同一范围内。

这 Trainer 将在多个 GPU 或 TPU 上开箱即用，并提供许多选项，例如混合精度训练（在训练的参数中使用 fp16 = True ）。我们将在第 10 章讨论它支持的所有内容。

使用Trainer API微调的介绍到此结束。对最常见的 NLP 任务执行此操作的示例将在第 7 章中给出，但现在让我们看看如何在纯 PyTorch 中执行相同的操作。

## 总结

进行数据处理，在 GLUE SST-2 数据集上微调模型。

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer


raw_datasets=load_dataset("glue","sst2")

checkpoint="bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=AutoModelForSequenceClassification.from_pretrained(checkpoint)
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(example):
    return tokenizer(example["sentence"],truncation=True)

tokenized_datasets=raw_datasets.map(tokenize_function,batched=True)

training_args=TrainingArguments("test-trainer",save_strategy="steps", save_steps=5000)
trainer = Trainer(
    model,    # 模型
    training_args,    #参数
    train_dataset=tokenized_datasets["train"],    #训练数据集
    eval_dataset=tokenized_datasets["validation"],    #测试数据集
    data_collator=data_collator,    #数据集线器
    tokenizer=tokenizer     #分词器
)

trainer.train()

In [ ]:
# logits形式的预测值
train_pred=trainer.predict(tokenized_datasets["validation"])
print(train_pred)
print(train_pred.predictions.shape, train_pred.label_ids.shape)
print(train_pred.metrics)

In [ ]:
import numpy as np

# 预测值，转换成y_label标签的形式
preds=np.argmax(train_pred.predictions,axis=-1)
print(preds)

In [ ]:
import evaluate

# 调用evaluate库进行评估
metric=evaluate.load("glue","sst2")
metric.compute(predictions=preds,references=predictions.label_ids)

In [ ]:
# 封装自己的评估函数
# 传入参数eval_preds指的是trainer.predict()输出的preds预测值
def compute_metrics(eval_preds):
    metric=evaluate.load("glue","sst2")
    logits,labels=eval_preds
    preds=np.argmax(logits,axis=-1)
    return metric.compute(predictions=preds,references=labels)

In [ ]:
training_args=TrainingArguments("text-trainer",evaluation_strategy="epoch")
model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

trainer=Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()